Camel 3 Function 
=====================


In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [3]:
# Constants and Parameters
const n, a, b = 2, 5, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.9,  8 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = camel # Objective function

camel (generic function with 1 method)

In [4]:
d = 6 # Initial Degree 
# SMPL = calculate_samples(binomial(n + d, d), delta, alpha)#Number of samples is too LinearAlgebra
SMPL = 200
println("Number of samples: ", SMPL)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

Number of samples: 200
current L2-norm: 1.2745075428821799e-14
Number of samples: 200
current L2-norm: 8.623940332309047e-15
Number of samples: 200


In [5]:
@polyvar(x[1:n]) # Define polynomial ring 

(Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}[x₁, x₂],)

In [7]:
real_pts_cheb = solve_polynomial_system(x, n, d, pol_cheb.coeffs; basis=:chebyshev, bigint=true)
real_pts_lege = solve_polynomial_system(x, n, d, pol_lege.coeffs; basis=:legendre, bigint=true)

# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Tracking 169 paths...   1%|▍                            |  ETA: 0:14:40



Tracking 169 paths... 100%|█████████████████████████████| Time: 0:00:11
  # paths tracked:                  169
  # non-singular solutions (real):  169 (19)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         169 (19)
Tracking 169 paths...   1%|▍                            |  ETA: 0:07:51



Tracking 169 paths...   2%|▋                            |  ETA: 0:04:05



Tracking 169 paths...   4%|█                            |  ETA: 0:02:49



Tracking 169 paths...   5%|█▍                           |  ETA: 0:02:11



Tracking 169 paths...   6%|█▊                           |  ETA: 0:01:47



Tracking 169 paths...   7%|██                           |  ETA: 0:01:32



Tracking 169 paths...   8%|██▍                          |  ETA: 0:01:21



Tracking 169 paths...   9%|██▊                          |  ETA: 0:01:12



Tracking 169 paths...  11%|███▏                         |  ETA: 0:01:05



Tracki

Row,x1,x2,z
,Float64,Float64,Float64
1,1.23023,0.162335,2.4963
2,-1.23023,-0.162335,2.4963
3,-1.63807,-0.228674,2.22936
4,1.90563e-15,5.65238e-14,-1.26575e-26
5,1.6071,0.568651,2.10425
6,1.10921,-0.768268,0.543719
7,-1.29607,-0.605084,2.22947
8,-1.6071,-0.568651,2.10425
9,0.089842,-0.712656,-1.03163


In [8]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_lege.grid
z_coords = pol_lege.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Camel 2d",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x1,
        scale_factor .* df_cheb.x2,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    scatter!(ax, scale_factor .* df_lege.x1,
        scale_factor .* df_lege.x2,
        df_lege.z,
        markersize=10,
        color=:yellow,
        label="Legendre approximant critical points")

    display(fig)
end

GLMakie.Screen(...)

In [7]:
# GLMakie.closeall()